# Assignment 4.1: Tilt Alarm System

## Part A4.1

### Buzzer sensor module used: KY006 (Passive) or KY012- (Active)

#### Include placeholder

In [4]:
import time
from pynq.lib import Pmod_IO
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
import socket
import multiprocessing

In [5]:
FAILURE = -1
SUCCESS = 0
SERVER_FREQ = 2500

In [6]:
#creat Pmod_IO object to be used for server. Wired to + of buzzer sensor to PMODB
gpio_pin_1 = Pmod_IO(base.PMODB,1,'out')
#create Pmod_IO object to be used for client. Wired to + of buzzer sensor to PMODA
gpio_pin_2 = Pmod_IO(base.PMODA,1,'out')

In [7]:
#store these pins in a dictionary to easily refrence based on server client
pins = {
    'server'   : gpio_pin_1, 
    'client'   : gpio_pin_2,
}

In [8]:
# definition to turn buzzer off
def turn_buzzer_off(index):
    pins[index].write(0)

In [9]:
# def function to test buzzer
def turn_buzzer_on(index):
    pins[index].write(1)

In [10]:
# def function to turn buzzer on for particular frequency
def turn_alarm_on(freq, index):
    if freq < 0:
        print("frequency should be > 0")
        return FAILURE
    if index not in pins:
        print("index passed does not exist {}".format(index))
        return FAILURE

    turn_buzzer_on(index)
    time.sleep((1/(2*freq)))
    turn_buzzer_off(index)
    time.sleep((1/(2*freq)))
    
    return SUCCESS

In [11]:
freq = 15000 #Hz
index = 'server'
count = 0
while count < 2000: 
    turn_alarm_on(freq, index)
    count +=1

In [12]:
from enum import Enum
class Button_State(Enum):
    Connect = 0
    Alarm = 1
    Disconnect = 2

Button_cur_State = Button_State.Connect

In [13]:
address = ('localhost', 6000)     # family is deduced to be 'AF_INET'

In [14]:
def server(_lock):
    global Button_cur_State
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 1: Bind the socket to the pynq board <CLIENT-IP> at port <LISTENING-PORT>
    # bind the socket to a public host, and a well-known port
    sock.bind(("127.0.0.1", 33334))
    
    #Always be a listener
    sock.listen(1)
    
    while True:
        # 2: Accept connections
        clientsocket, addr = sock.accept()
        print ('Got connection from', addr )
        
        # 3: Receive bytes from the connection 
        msg = clientsocket.recv(1024).decode()
        print ("Message received {}".format(msg.decode()))
        
        if msg == 'sound_alarm':
            #use lock
            _lock.acquire(True)
            sound_alarm = Button_cur_State
            _lock.release()
            
        while sound_alarm == Button_State.Alarm:
            turn_alarm_on(SERVER_FREQ, 'server')
            _lock.acquire(True)
            sound_alarm = Button_cur_State
            _lock.release()


In [15]:
def client(_lock):
    global Button_cur_State
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect(('127.0.0.1', 33334))
    
    cond = True
    
    while cond:
        _lock.acquire(True)
        Button_State = Button_cur_State
        _lock.release()
        if Button_State == Button_State.Alarm:
            sock.send('sound_alarm'.encode())
        elif Button_State == Button_State.Disconnect:
            cond = False
            sock.close()

In [17]:
import asyncio
btn_cond = True

async def get_btns(_loop,btns):
    global btn_cond, Button_cur_State
    while btn_cond:
        await asyncio.sleep(0.01)
        btn = btns.read()
        if btn == 0x01:
            Button_cur_State = Button_State.Connect
            print("Connect button pressed")
        elif btn == 0x02:
            Button_cur_State = Button_State.Alarm
            print("Alarm button pressed")
        elif btn == 0x04:
            Button_cur_State = Button_State.Disconnect
            print("Disconnect button pressed")
            btn_cond = False
            _loop.stop()
        #else:
            #print("Invalid button pressed")
            
#loop = asyncio.new_event_loop()
#loop.create_task(flash_leds())
#loop.create_task(get_btns(loop))
#loop.run_forever()
#loop.close()        

In [18]:
procs = [] # a future list of all our processes


In [19]:
import threading
import os
def main():
    
    btns = base.btns_gpio
    lock_bp = multiprocessing.Lock()
    #create a asyncio loop event 
    loop = asyncio.new_event_loop()
    loop.create_task(get_btns(loop,btns))
    
    # Launch process1 on CPU0
    p1_start = time.time()

    p1 = multiprocessing.Process(target=server, args=(lock_bp,)) 
    os.system("taskset -p -c {} {}".format(0, p1.pid)) 
    p1.start() # start the process
    procs.append(p1)

    # Launch process2 on CPU1
    p2_start = time.time()
    p2 = multiprocessing.Process(target=client, args=(lock_bp,)) 
    os.system("taskset -p -c {} {}".format(1, p2.pid))
    p2.start() # start the process
    procs.append(p2)
    
    p1Name = p1.name # get process1 name
    p2Name = p2.name # get process2 name
    
    loop.run_forever()
    loop.close()
    
    # Here we wait for process1 to finish then wait for process2 to finish
    p1.join() # wait for process1 to finish
    print('Process 1 with name, {}, is finished'.format(p1Name))

    p2.join() # wait for process2 to finish
    print('Process 2 with name, {}, is finished'.format(p2Name))
    

In [ ]:
if __name__ == '__main__':
    main()

Got connection from ('127.0.0.1', 46874)
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
Alarm button pressed
